# LT5-LE7 SR Fractional Abundances with Global Endmembers

In [49]:
import ee
from IPython.display import Image
# Example iamge display:
# image = ee.Image('srtm90_v4')
# Image(url=image.getThumbUrl({'min':0, 'max': 3000}))

ee.Initialize()

# Detroit MSA bounds
detroit_msa = ee.FeatureCollection('ft:1QZFRMLeORVfc9sYNOOKD2WyOlEyI8gmj-g3CdTlH', 'geometry')

# Water mask
water = ee.Image(1).where(ee.Image('JRC/GSW1_0/GlobalSurfaceWater').select('max_extent'), ee.Image(0))

# No other way, apparently, to get a range as a list; can't map type coercion client-side...
lt5_dates = ee.List([
    '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006',
    '2007', '2008', '2009', '2010', '2011'
])

le7_dates = ee.List([
    '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012',
    '2013', '2014', '2015'
])

combined_dates = ee.List([
    '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006',
    '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015'
])

# Look at 2013, 2013, and 2015 CDL and choose 1 where pixel is always non-cultivated, zero otherwise
# Cultivated = 2 in the "cultivated" band
cdl = ee.ImageCollection('USDA/NASS/CDL')\
    .filterBounds(detroit_msa)\
    .filter(ee.Filter([
        ee.Filter.inList('system:index', ee.List(['2013', '2014', '2015']))
    ]))\
    .map(lambda img: img.select('cultivated').lt(2))\
    .reduce(ee.Reducer.allNonZero())\
    .clip(ee.Feature(detroit_msa.union().toList(1).get(0)).bounds().geometry())

## Functions

In [66]:
def annual_vegetation_maximum(collection, year):
    '''
    Creates an annual vegetation composite, given the total collection of images
    and the year in which a subset of them should be composited.
    '''
    return ee.ImageCollection(collection.filterMetadata('year', 'equals', year))\
        .map(shade_normalize)\
        .select('V')\
        .max()\
        .set({'year': year})

    
def cloud_mask(img):
    '''
    Applies a cloud mask to an image based on the QA band of the
    Landsat data.
    '''
    # Bits 3 and 5 are cloud shadow and cloud, respectively
    cloud_shadow_bit_mask = ee.Number(2).pow(3).int()
    clouds_bit_mask = ee.Number(2).pow(5).int()
    water_mask = ee.Number(2).pow(2).int() # Bit 2 is water

    # Get the pixel QA band
    qa = img.select('pixel_qa')

    # Both flags should be set to zero, indicating clear conditions
    mask = qa.bitwiseAnd(cloud_shadow_bit_mask).eq(0)\
        .And(qa.bitwiseAnd(clouds_bit_mask).eq(0))\
        .And(qa.bitwiseAnd(water_mask).eq(0))\
        .reduceNeighborhood(ee.Reducer.anyNonZero(), ee.Kernel.square(1))
    # NOTE: Also mask those pixels that are adjacent to cloud, cloud shadow, or water

    # Return the masked image, scaled to [0, 1]
    return img.updateMask(mask).divide(10000)\
        .select(
            ['B1', 'B2', 'B3', 'B4', 'B5', 'B7', 'pixel_qa'],
            ['blue', 'green', 'red', 'NIR', 'SWIR1', 'SWIR2', 'pixel_qa'])\
        .set({
            'date': img.get('SENSING_TIME'),
            'year': ee.String(img.get('SENSING_TIME')).slice(0, 4)
        })


def lsma(img, ems, emv, emd):
    '''
    Constrain to be positive, sum-to-one, because these add information, 
    thereby increasing accuracy.
    '''
    return img.select('blue', 'green', 'red', 'NIR', 'SWIR1', 'SWIR2')\
        .unmix([ems, emv, emd], True, True).rename(['S', 'V', 'D'])\
        .set({'year': img.get('year')})


def svd_unmixing_etm_plus(img):
    '''
    Spectral mixture analysis using Sousa & Small's revised global
    endmembers; tables are in Supplemental Data as text files. Reference:
        Sousa, D., and C. Small. 2017. 
        Global cross-calibration of Landsat spectral mixture models. 
        Remote Sensing of Environment 192:139–149.
        
    These endmembers are for ETM+ surface reflectance data; should work 
    for TM surface reflectance data as well.
    '''
    ems = [0.218413, 0.344440, 0.535987, 0.669174, 0.754645, 0.671738]
    emv = [0.100880, 0.098638, 0.067241, 0.585458, 0.208614, 0.088058]
    emd = [0.083704, 0.047546, 0.023937, 0.010864, 0.003250, 0.002208]
    return lsma(img, ems, emv, emd)


def shade_normalize(img):
    '''
    Returns a shade-normalized fractional abundance for substrate
    and vegetation.
    '''
    substrate = img.select('S')
    vegetation = img.select('V')
    return substrate.addBands(vegetation).divide(substrate.add(vegetation))

## Assets

In [76]:
lt5_collection = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR')\
    .filterDate('1995-01-01', '2015-12-31')\
    .filter(ee.Filter.dayOfYear(121, 273))\
    .filterBounds(detroit_msa)\
    .map(cloud_mask)\
    .map(lambda img: img.select('blue', 'green', 'red', 'NIR', 'SWIR1', 'SWIR2'))

le7_collection = ee.ImageCollection('LANDSAT/LE07/C01/T1_SR')\
    .filterDate('1995-01-01', '2015-12-31')\
    .filter(ee.Filter.dayOfYear(121, 273))\
    .filterBounds(detroit_msa)\
    .map(cloud_mask)\
    .map(lambda img: img.select('blue', 'green', 'red', 'NIR', 'SWIR1', 'SWIR2'))

# Compute SVD fractional abundance images
lt5_svd = lt5_collection.map(lambda img: svd_unmixing_etm_plus(img))
le7_svd = le7_collection.map(lambda img: svd_unmixing_etm_plus(img))
svd_collection = ee.ImageCollection(lt5_svd).merge(le7_svd)

## Workspace

In [ ]:
# Compute the max vegetation abundance in each band, in each pixel
composites = ee.ImageCollection(combined_dates
    .map(lambda year: annual_vegetation_maximum(svd_collection, year)))\
    .map(lambda img: img.clip(detroit_msa)) # To get around "unable to export unbounded image"

for i in xrange(0, ee.ImageCollection(composites).size().getInfo()):
    img = ee.Image(ee.ImageCollection(composites).toList(1, i).get(0))
    task = ee.batch.Export.image.toDrive(image = img, 
        description = 'SVD_Vegetation_Maximum_%s' % img.get('year'), 
        region = detroit_msa.geometry().bounds().getInfo()['coordinates'],
        folder = 'EarthEngine', scale = 30, crs = 'EPSG:32617')
    task.start()